# Demo Notebook to use SSLimPy 

In this notebook we will show how to set up SSLimPy,\
Compute the LIM power spectra for a particular case,\
And do a quick 2 parameter Fisher forecast

## Initial Setup

In [ ]:
import os
import sys

sys.path.append("../")

envkey = "OMP_NUM_THREADS"
# Set this environment variable to the number of available cores in your machine,
# to get a fast execution of the Einstein Boltzmann Solver
print("The value of {:s} is: ".format(envkey), os.environ.get(envkey))
os.environ[envkey] = str(12)
print("The value of {:s} is: ".format(envkey), os.environ.get(envkey))

In [ ]:
import astropy.units as u
import matplotlib.pyplot as plt
from copy import copy, deepcopy
import numpy as np
import seaborn
from getdist.gaussian_mixtures import GaussianND
from getdist import plots

In [ ]:
Cs = seaborn.color_palette("colorblind")
Cs

## Choose model parameters and save them in dictionaries

In [ ]:
# Settings will be the global settings that should not change during a given run.
# For the full list see the Configuration class in interface

settings = {
    "code":"camb", # The Einstein--Boltzman solver that should be used
    "do_RSD" : True, # If RSD should be considerd
    "nonlinearRSD" : True, # If you want to add FOG to the RSD
    "QNLpowerspectrum": True, # Use dewiggled power spectrum (vlasov approximation of nonlinear structure formation)
    "FoG_damp" : "ISTF_like", # The particular parametrization for the FOG. Check PowerSpectrum for the full list
    "halo_model_PS" : False, # If the cosmological shotnoise should be computed from the halo model 
    "output" : ["Power spectrum", "Covariance"], # What output one wants (here power spectrum and Gaussian covariance only)
    "kmin": 1e-4 * u.Mpc**-1,
    "kmax": 50 * u.Mpc**-1,
    "nk": 200,
}

In [ ]:
# Cosmological parameters for the fiducial cosmology can be set like this
# Depending on your test you can fix them vary them
cosmodict={
    "h": 0.6737,
    "Omegam": 0.3146,
    "Omegab": 0.0492,
    "As":2.1e-9,
    "ns":0.966,
    "mnu":0.06,
    "Neff":3.044,
}

In [ ]:
# Parameters that enter your halo model. Typically they are not changed but you could
halodict={
    "halo_tracer" : "clustering", # Computes all halo quantities from the matter field - neutrinos
    "hmf_model": "ST", # Sheth--Tormann halo mass function
    "concentration": "Diemer19", # Diemer19 halo concentration relation
    "bias_model": "ST99",
    "bias_pars": {
        "q" : 0.707,
        "p" : 0.3,
    },
}

In [ ]:
# Parameters for Astrophysics.
astrodict={
    "model_type": "ML",
    "model_name": "SilvaCII",
    "model_par": {
        "a": 0.939,
        "b": 6.639,
        "SFR_file": "Fonseca16_Lya_SFR_params.dat",
        "do_quench": True,
    },
    "sigma_scatter" : 0.5
}

In [ ]:
# Parameters for the Survey specifications
surveyspecs = {
        "Tsys_NEFD": 40 * u.uK, #System temperature for instrumental shotnoise
        "Nfeeds": 19,
        "tobs": 1300 * u.h,
        "nD": 1, # Observational parameters
        "beam_FWHM": 0.5 * u.arcmin,
        "nu":  1.897 * u.THz, # CII
        "dnu": 900 * u.MHz, # Spectrograph resolution
        "nuObs": 250 * u.GHz, # Observed Frequency
        "Delta_nu": 50 * u.GHz, # Frequency Bin
        "Omega_field": 140 * u.deg**2, # Angular size of survey
}

## Compute instance of SSLimPy to work with

In [ ]:
# Import Main modules. This might take some time as some functions compile before time
from SSLimPy.interface import sslimpy
from SSLimPy.cosmology import cosmology
from SSLimPy.cosmology import halo_model
from SSLimPy.cosmology import astro

In [ ]:
myssl = sslimpy.SSLimPy(
    settings_dict=settings,
    cosmopars=cosmodict,
    halopars=halodict,
    astropars=astrodict,
    obspars_dict=surveyspecs,
)

The SSLimPy class can be used directly as a cosmology calculator

In [ ]:
z = np.linspace(0, 5)

fig, axs = plt.subplots(1, 2, figsize=(12, 5))

H = myssl.current_cosmology.Hubble(z, physical=True).to(u.km * u.s**-1 * u.Mpc**-1)
D_A = myssl.current_cosmology.angdist(z)

axs[0].plot(z, H, c=Cs[0])
axs[1].plot(z, D_A, c=Cs[1])

axs[0].set_xlabel(r"$z$")
axs[1].set_xlabel(r"$z$")
axs[0].set_ylabel(r"$H(z)\,[\mathrm{km}\,\mathrm{s}^{-1}\,\mathrm{Mpc}^{-1}]$")
axs[1].set_ylabel(r"$D_\mathrm{A}(z)\,[\mathrm{Mpc}]$")

We could now compute the LIM power spectrum using the PowerSpectrum Class

In [ ]:
from SSLimPy.LIMsurvey import power_spectrum

In [ ]:
# The power spectrum can be computed on a different grid than the internal quantities of SSLimPy
# The internal grid is for numerical stability while this grid can be tailored to your survey
pobs_settings = {
    "kmin" : 1e-3 * u.Mpc**-1,
    "kmax" : 5 * u.Mpc**-1,
    "nk" : 99,
    "k_kind": "log",
    "nmu" : 128,
}

# The bias could be computed from the halo model or passed as a free parameter
BAOdict = {
    "bmean" : 2.8,
}

pobs = power_spectrum.PowerSpectra(myssl.current_astro, BAOpars=BAOdict, settings=pobs_settings)

In [ ]:
# Plot the monopole and quadrupole of LIM
k = pobs.k
Pk_0bs = pobs.Pk_0bs
Pk_2bs = pobs.Pk_2bs

plt.loglog(k, k[:, None] * Pk_0bs, c=Cs[0], label="Monopole")
plt.loglog(k, k[:, None] * Pk_2bs, c=Cs[1], label="Quadrupole")
plt.loglog(k, - k[:, None] * Pk_2bs, c=Cs[1],ls="--")
plt.xlabel(r"$k\,[\mathrm{Mpc}^{-1}]$")
plt.ylabel(r"$k\,P^\mathrm{TT}\,[\mu\,\mathrm{K}^2\,\mathrm{Mpc}^{2}]$")
plt.legend()

Or one could compute the Gaussian covariance as well

In [ ]:
from SSLimPy.LIMsurvey import covariance

In [ ]:
cov = covariance.Covariance(pobs)
Gaussian_cov = cov.gaussian_cov()

In [ ]:
plt.loglog(k, Gaussian_cov[:, 0, 0, 0], c=Cs[0], label = r"$C_{00}(k)$")
plt.loglog(k, Gaussian_cov[:, 1, 0, 0], c=Cs[1], label = r"$C_{20}(k)$")
plt.loglog(k, -Gaussian_cov[:, 1, 0, 0], ls="--", c=Cs[1])
plt.loglog(k, Gaussian_cov[:, 1, 1, 0], c=Cs[2], label = r"$C_{22}(k)$")
plt.xlabel(r"$k\,[\mathrm{Mpc}^{-1}]$")
plt.ylabel(r"$C^\mathrm{TT}_{\ell \ell}\,[\mu\,\mathrm{K}^4\,\mathrm{Mpc}^{6}]$")
plt.legend()

## Perform a very basic Fisher forecast

An instance of SSLimPy is mainly interacted with through the compute function.\
If no specific output is asked for will compute whatever was asked for in settings. \
One can pass new model parameters as well but for now we will compute the fiducial value.

In [ ]:
# Helper function to copy dictionaries from fiducials
def obtain_dicts(ssl: sslimpy.SSLimPy):
    varycosmodict = copy(ssl.fiducialcosmoparams)
    varyhalodict = copy(ssl.fiducialhaloparams)
    varyspecdict = copy(ssl.fiducialspecparams)
    varyastrodict = deepcopy(ssl.fiducialastroparams) # The model_par dictionarry is nested in the astro dict
    varyBAOdict = copy(BAOdict)

    return varycosmodict, varyhalodict, varyspecdict, varyastrodict, varyBAOdict

For this demonstration we vary:\
&emsp; two cosmo parameters,\
&emsp; the amblitude of clustering,\
&emsp; and the shape of the mass luminosty relation

In [ ]:
# choose which parameters you want to vary and the relative stepsize
freepars = {"h" : 0.01, "Omegam": 0.01, "bmean" : 0.01, "a": 0.01, "b": 0.01} 

derivdict = {}
means = {}
for par in freepars:
    eps = freepars[par]

    # Compute the power spectrum for a posive step

    # get copies of fiducial dictionaries
    varycosmodict, varyhalodict, varyspecdict, varyastrodict, varyBAOdict = obtain_dicts(myssl)
    varyastropardict = varyastrodict["model_par"]

    # update the dictionaries for a free parameter
    dictlist = [varycosmodict, varyhalodict, varyspecdict, varyastropardict, varyBAOdict]
    for updatedict in dictlist:
        if par in updatedict:
            fidpar = updatedict[par]
            means[par] = fidpar
            ppar =  fidpar * (1 + eps)
            updatedict[par] = ppar
    varyastrodict["model_par"] = varyastropardict

    presults = myssl.compute(
        varycosmodict, varyhalodict, varyastrodict, varyspecdict, 
        BAOpars=varyBAOdict, pobs_settings=pobs_settings,
        output=["Power spectrum"],
    ) # The compute function returns a dictionary of the asked for output
    ppobs = presults["Power spectrum"] # The only output is the Power spectrum in this case
    
    # Compute the power spectrum for a negative step

    varycosmodict, varyhalodict, varyspecdict, varyastrodict, varyBAOdict = obtain_dicts(myssl)
    varyastropardict = varyastrodict["model_par"]

    dictlist = [varycosmodict, varyhalodict, varyspecdict, varyastropardict, varyBAOdict]
    for updatedict in dictlist:
        if par in updatedict:
            mpar = updatedict[par] * (1 - eps)
            updatedict[par] = mpar
    varyastrodict["model_par"] = varyastropardict

    mresults = myssl.compute(
        varycosmodict, varyhalodict, varyastrodict, varyspecdict, 
        BAOpars=varyBAOdict, pobs_settings=pobs_settings,
        output=["Power spectrum"],
    )
    mpobs = mresults["Power spectrum"]

    P0_deriv = np.squeeze(ppobs.Pk_0bs - mpobs.Pk_0bs) / (2 * eps * fidpar)
    P2_deriv = np.squeeze(ppobs.Pk_2bs - mpobs.Pk_2bs) / (2 * eps * fidpar)
    
    derivdict[par] = np.array([*P0_deriv.value, *P2_deriv.value]) * P0_deriv.unit

We now want to construct the covariance for our stacked observable  

In [ ]:
C00 = np.diag(Gaussian_cov[:, 0, 0, 0])
C20 = np.diag(Gaussian_cov[:, 1, 0, 0])
C02 = np.diag(Gaussian_cov[:, 0, 1, 0])
C22 = np.diag(Gaussian_cov[:, 1, 1, 0])

stacked_Gaussian_cov = np.block([[C00, C02], [C20, C22]])
stacked_inv_cov = np.linalg.inv(stacked_Gaussian_cov)

In [ ]:
freepar_names = np.array([*freepars.keys()])
Fij = np.zeros((*freepar_names.shape, *freepar_names.shape))

In [ ]:
freepar_names_to_latex = {
    "Omegam" : r"\Omega_\mathrm{m}",
    "h": r"h",
    "bmean": r"\langle b \rangle",
    "a": r"a_\mathrm{ML}",
    "b": r"b_\mathrm{ML}"
}
latex_names = [freepar_names_to_latex[par] for par in freepar_names]

In [ ]:
mean = []
for i, par1 in enumerate(freepar_names):
    mean.append(means[par1])
    for j, par2 in enumerate(freepar_names):
        Fij[i,j] = np.einsum("i,ij,j", derivdict[par1], stacked_inv_cov, derivdict[par2])

In [ ]:
gauss1=GaussianND(mean=mean, cov=Fij, names=freepar_names, is_inv_cov=True, labels=latex_names)

# Set default font sizes before creating the plotter
plot_settings = plots.GetDistPlotSettings()
plot_settings.axes_fontsize = 24
plot_settings.axes_labelsize = 26
plot_settings.legend_fontsize = 24

g = plots.get_subplot_plotter(subplot_size_ratio=1)
g.triangle_plot([gauss1], filled=[False], params=freepar_names, legend_labels=[r'SSLimPy Demo'],contour_lws=[1])

In [ ]:
invFisher = np.linalg.inv(Fij)
dinvFisher = np.diag(invFisher)
Corr = invFisher / np.sqrt(np.outer(dinvFisher, dinvFisher))

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
cax = ax.imshow(Corr, cmap='coolwarm', vmin=-1, vmax=1)

cbar = fig.colorbar(cax, ax=ax)
cbar.set_label('Correlation Coefficient')

# Set parameter names as tick labels
ax.set_xticks(np.arange(len(freepar_names)))
ax.set_yticks(np.arange(len(freepar_names)))

latex_names2 = ["$" + ln +"$" for ln in latex_names]
ax.set_xticklabels(latex_names2)
ax.set_yticklabels(latex_names2)

# Add the correlation values as text
for i in range(len(freepar_names)):
    for j in range(len(freepar_names)):
        text = f"{Corr[i, j]:.2f}"
        ax.text(j, i, text, ha='center', va='center', color='black' if abs(Corr[i, j]) < 0.75 else 'white')

ax.set_title("Parameter Correlation Matrix")
plt.tight_layout()
plt.show()